In [4]:
import pandas as pd
import mysql.connector
import sys
import os

In [6]:
credentials_path = r'E:\StuDY\FIT5120\db_keys'
sys.path.append(credentials_path)
from db_keys import mysql_host, mysql_user, mysql_password, mysql_database

import pymysql

# Connect to your MySQL database

mydb = mysql.connector.connect(
    host=mysql_host,
    user=mysql_user,
    password=mysql_password,
    database=mysql_database
)

mycursor = mydb.cursor()

In [18]:
df  = pd.read_excel('Gender pay gap measures.xlsx')
df.head()

,Date,Median hourly cash earnings (EEH) (%),Mean hourly cash earnings (EEH) (%),Median weekly cash earnings (EEH) (%),Mean weekly cash earnings (AWE) (%),Mean weekly ordinary time cash earnings of full-time adult employees (AWE)(a) (%),Commonly cited measure (AWE)(a) (%)
0,May-14,12.8,16.1,31.2,34.2,17.1,18.3
1,Aug-14,NaN,NaN,NaN,NaN,NaN,NaN
2,Nov-14,NaN,NaN,NaN,34.2,17.3,18.6
3,Feb-15,NaN,NaN,NaN,NaN,NaN,NaN
4,May-15,NaN,NaN,NaN,32.6,16.6,17.9


In [19]:
df = df[['Date', 'Mean weekly cash earnings (AWE) (%)',  'Commonly cited measure (AWE)(a) (%)' ,
         'Mean weekly ordinary time cash earnings of full-time adult employees (AWE)(a) (%)']]
df = df.dropna()
df = df.rename(columns = {'Mean weekly cash earnings (AWE) (%)': 'Mean weekly cash earnings(%)', 
                     'Commonly cited measure (AWE)(a) (%)': 'Commonly cited measure(%)',
                     'Mean weekly ordinary time cash earnings of full-time adult employees (AWE)(a) (%)' : 'Mean weekly cash earnings of full-time adult employees(%)'})
df1 = df[['Date', 'Mean weekly cash earnings(%)']].rename(columns = {'Mean weekly cash earnings(%)':'percentage'})
df1['type'] = 'Mean weekly cash earnings(%)'
df2 = df[['Date', 'Commonly cited measure(%)']].rename(columns = {'Commonly cited measure(%)':'percentage'})
df2['type'] = 'Commonly cited measure(%)'
df3 = df[['Date', 'Mean weekly cash earnings of full-time adult employees(%)']].rename(columns = {'Mean weekly cash earnings of full-time adult employees(%)':'percentage'})
df3['type'] = 'Mean weekly cash earnings of full-time adult employees(%)'


In [ ]:
try:

    # Insert individual data
    for index, row in df.iterrows():
        sql = f"""
        INSERT INTO pay_gap (record_date, percentage_type,  percentage)
        VALUES ('{row['Date']}', '{row['type']}', {row['percentage']});
        """
        mycursor.execute(sql)

    mydb.commit()

finally:
    mydb.close()

In [20]:
df = pd.concat([df1,df2,df3],axis = 0)
df.head()

,Date,percentage,type
0,May-14,34.2,Mean weekly cash earnings(%)
2,Nov-14,34.2,Mean weekly cash earnings(%)
4,May-15,32.6,Mean weekly cash earnings(%)
6,Nov-15,32.2,Mean weekly cash earnings(%)
8,May-16,32.5,Mean weekly cash earnings(%)


In [21]:
df.to_excel('pay_gap.xlsx', index=False)